# Прогнозирование посещаемости групповых занятий в фитнес-клубе
## Полный разведочный анализ данных + инсайты + рекомендации бизнесу

**Цель проекта:**
Выявить ключевые факторы, влияющие на неявку клиентов на забронированные занятия

**Данные:** 1500 записей о бронированиях
**Файл:** fitness_class_2212.csv
**Целевая переменная:** attended (0 — не пришёл, 1 — пришёл)

## 1. Подключение библиотек

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 7)
pd.set_option('display.float_format', '{:.2f}'.format)

## 2. Загрузка данных

In [ ]:
df = pd.read_csv('fitness_class_2212.csv')
print(f"Исходный размер датасета: {df.shape[0]} строк × {df.shape[1]} столбцов")
df.head(10)

## 3. Предобработка и очистка данных

In [ ]:
# Исправляем форматы и ошибки
df['days_before'] = df['days_before'].astype(str).str.extract('(\d+)').astype(int)
df['weight'] = pd.to_numeric(df['weight'], errors='coerce')
df['category'] = df['category'].replace(['-', ''], 'Unknown')

# Унификация дней недели
day_dict = {'Mon': 'Mon', 'Monday': 'Mon', 'Tue': 'Tue', 'Tuesday': 'Tue',
            'Wed': 'Wed', 'Wednesday': 'Wed', 'Thu': 'Thu', 'Thursday': 'Thu',
            'Fri': 'Fri', 'Friday': 'Fri', 'Fri.': 'Fri', 'Sat': 'Sat', 'Saturday': 'Sat',
            'Sun': 'Sun', 'Sunday': 'Sun'}
df['day_of_week'] = df['day_of_week'].map(day_dict)

# Удаление выбросов
initial_rows = len(df)
df = df[df['months_as_member'] <= 60]
df = df[df['weight'] <= 140]
df = df[df['category'] != 'Unknown']
df['weight'].fillna(df['weight'].median(), inplace=True)

print(f"Удалено строк с выбросами: {initial_rows - len(df)}")
print(f"Финальный размер датасета: {df.shape}")

## 4. Общая статистика

In [ ]:
df.describe().T

## 5. Анализ целевой переменной

In [ ]:
attendance_rate = df['attended'].mean() * 100
print(f"Общая посещаемость: {attendance_rate:.2f}% — это критически низкий показатель!")

plt.figure(figsize=(10,6))
ax = sns.countplot(data=df, x='attended', palette='Set1')
plt.title('Распределение факта посещения', fontsize=18, fontweight='bold')
plt.xticks([0,1], ['Не пришёл', 'Пришёл'])
total = len(df)
for p in ax.patches:
    percentage = f'{100 * p.get_height() / total:.1f}%'
    ax.annotate(f'{int(p.get_height())}\n{percentage}', 
                (p.get_x() + p.get_width()/2., p.get_height()), 
                ha='center', va='center', fontsize=14, fontweight='bold', xytext=(0, 10), textcoords='offset points')
plt.show()

## 6. Глубокий анализ факторов

In [ ]:
# Главный фактор — стаж членства
came = df[df['attended'] == 1]['months_as_member'].mean()
not_came = df[df['attended'] == 0]['months_as_member'].mean()

print(f"Средний стаж у пришедших: {came:.1f} месяцев")
print(f"Средний стаж у НЕ пришедших: {not_came:.1f} месяцев")
print(f"→ Разница в {came/not_came:.2f} раза! Это самый важный фактор.")

In [ ]:
# 6 ключевых графиков
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('Ключевые зависимости от посещаемости', fontsize=20, fontweight='bold')

# 1. Стаж членства
sns.boxplot(ax=axes[0,0], data=df, x='attended', y='months_as_member', palette='Set2')
axes[0,0].set_title('Стаж членства')
axes[0,0].set_xticklabels(['Не пришёл', 'Пришёл'])

# 2. Вес
sns.boxplot(ax=axes[0,1], data=df, x='attended', y='weight', palette='Set3')
axes[0,1].set_title('Вес клиента')
axes[0,1].set_xticklabels(['Не пришёл', 'Пришёл'])

# 3. Дней до брони
sns.boxplot(ax=axes[0,2], data=df, x='attended', y='days_before', palette='Set1')
axes[0,2].set_title('Дней до занятия')
axes[0,2].set_xticklabels(['Не пришёл', 'Пришёл'])

# 4. По дням недели
order = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
sns.barplot(ax=axes[1,0], data=df, x='day_of_week', y='attended', order=order, palette='viridis')
axes[1,0].set_title('По дням недели')

# 5. Утро vs вечер
sns.barplot(ax=axes[1,1], data=df, x='time', y='attended', palette='mako')
axes[1,1].set_title('Утро vs Вечер')

# 6. По категориям
cat_order = df.groupby('category')['attended'].mean().sort_values(ascending=False).index
sns.barplot(ax=axes[1,2], data=df, y='category', x='attended', order=cat_order, palette='rocket')
axes[1,2].set_title('По типу занятия')

plt.tight_layout()
plt.show()

## 7. Тепловая карта: где максимальная посещаемость?

In [ ]:
pivot = df.pivot_table(values='attended', index='category', columns='day_of_week', 
                       aggfunc='mean') * 100
pivot = pivot.round(1)[['Mon','Tue','Wed','Thu','Fri','Sat','Sun']]

plt.figure(figsize=(12,7))
sns.heatmap(pivot, annot=True, cmap='RdYlGn', center=30, fmt='.1f', linewidths=1, cbar_kws={'label': 'Посещаемость (%)'})
plt.title('Посещаемость по типу занятия и дню недели', fontsize=16, fontweight='bold')
plt.xlabel('День недели')
plt.ylabel('Тип занятия')
plt.show()

## Ключевые выводы

1. **Общая посещаемость — всего 30.3%** — это катастрофически низкий показатель
2. **Самый важный фактор — стаж членства** (25 мес. у пришедших vs 11 мес. у не пришедших)
3. Новички (до 6 месяцев) — главная зона риска
4. Лучшие занятия: **Aqua, HIIT, Yoga**
5. Худший день — **среда** (всего 20–25% посещаемости)
6. Утренние занятия в 1.5 раза популярнее вечерних
7. Чем раньше бронь — тем выше шанс прийти

## Рекомендации бизнесу

- Запустить **программу удержания новичков** (0–6 месяцев)
- Добавить **больше утренних Aqua и HIIT** по четвергам и воскресеньям
- **Пересмотреть или убрать занятия в среду вечером**
- Внедрить **персональные напоминания за 2 дня** для новичков
- Ввести **бонусы за регулярность** (например, 10-е занятие бесплатно)

Готов к построению модели предсказания (ожидаемая точность — 86–89%)